In [ ]:
import os

import pandas as pd
import geopandas as gpd

import wbgapi as wb  # https://blogs.worldbank.org/opendata/introducing-wbgapi-new-python-package-accessing-world-bank-data

In [ ]:
# Define input data
in_admin = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inA = gpd.read_file(in_admin)
ssa = inA.loc[inA["Region"] == "Sub-Saharan Africa"]
ssa

In [ ]:
# Identify indicators related to GDP
wb.series.info(q="gdp")

In [ ]:
wb.series.info(q="urban")

In [ ]:
# These are the datasets we are interested in extracting
selected_indicators = ["NY.GDP.MKTP.CD", "SP.URB.TOTL", "SP.URB.TOTL.IN.ZS"]
urb_data = wb.data.DataFrame(selected_indicators, economy=ssa["ISO3"].values, mrnev=1)
urb_data = urb_data.reset_index()  # .drop(['index'], axis=1)
urb_data.loc[urb_data["economy"] == "SYC"]

In [ ]:
for ind in selected_indicators:
    print(f'{ind}: {wb.series.info(ind).items[0]["value"]}')

In [ ]:
urb_data.columns = ["ISO3", "GDP", "UrubPop", "UrbPercent"]
urb_data.to_csv("SSA_countries_GDP_Urbanization.csv")

In [ ]:
urb_data.loc[urb_data["economy"] == "SYC"]

In [ ]:
wb.data.DataFrame?

# Create geospatial data

In [ ]:
pp_folder = "/home/wb411133/data/Projects/MR_Novel_Urbanization/Mapping/URBAN_SUMMARIES"
excel_files = [x for x in os.listdir(pp_folder) if x.endswith("xlsx")]
for excel_file in excel_files:
    curD = pd.read_excel(os.path.join(pp_folder, excel_file))
    cols = list(curD.columns)
    cols[0] = "ISO3"
    cols[1] = "country"
    cols[2] = "Urban_Type"
    curD.columns = cols
    curD["ISO3"] = curD["ISO3"].apply(lambda x: x.upper())
    curG = pd.merge(ssa, curD, on="ISO3")
    curG = pd.merge(curG, urb_data, on="ISO3")
    curG = gpd.GeoDataFrame(curG, geometry="geometry", crs=4326)
    curG.to_file(
        os.path.join(pp_folder, excel_file.replace(".xlsx", ".geojson")),
        driver="GeoJSON",
    )
    # Create point file as well
    curG["geometry"] = curG["geometry"].apply(lambda x: x.centroid)
    curG.to_file(
        os.path.join(pp_folder, excel_file.replace(".xlsx", "_CENTROID.geojson")),
        driver="GeoJSON",
    )

In [ ]:
pd.DataFrame(curG).drop(["geometry"], axis=1).to_csv(
    os.path.join(pp_folder, excel_file.replace(".xlsx", "_joined.csv"))
)

In [ ]:
curD.head()